# mdxplain Quickstart Tutorial: Introduction to Conformational Analysis with mdxplain

In [ ]:
mdxplain_root = "../"
data_path = "../data/2RJY/"

In [ ]:
from mdxplain import PipelineManager

In [ ]:
# Initialize pipeline and load trajectory data
pipeline = PipelineManager(use_memmap=True, chunk_size=1000, show_progress=False)

pipeline.trajectory.load_trajectories(data_input=str(data_path))
pipeline.trajectory.add_labels(
        traj_selection="all"
    )

In [ ]:
pipeline.feature.add.distances()
pipeline.feature.add.contacts(cutoff=4.5)
pipeline.feature_selector.create("contacts_only")
pipeline.feature_selector.add.contacts("contacts_only", "all")
pipeline.feature_selector.select("contacts_only")

In [ ]:
pipeline.decomposition.add.contact_kernel_pca(
    n_components=4,
    selection_name="contacts_only", 
    decomposition_name="ContactKernelPCA",
)

In [ ]:
pipeline.clustering.add.dpa(
    "ContactKernelPCA", 
    Z=2.5,
    cluster_name="DPA_ContactKPC",
)

In [ ]:
fig = pipeline.plots.clustering.membership(
    clustering_name="DPA_ContactKPC",
    save_fig=True,
    file_format="svg",
    tick_fontsize=18,
    xlabel_fontsize=18,
    ylabel_fontsize=18,
)

In [ ]:
fig = pipeline.plots.landscape(
    decomposition_name="ContactKernelPCA",
    dimensions=[0, 1, 2, 3],
    clustering_name="DPA_ContactKPC",
    save_fig=True,
    file_format="svg",
    tick_fontsize=26,
    xlabel_fontsize=26,
    ylabel_fontsize=26,
    contour_label_fontsize=26,
)

In [ ]:
pipeline.data_selector.create_from_clusters(
    group_name="cluster",
    clustering_name="DPA_ContactKPC"
)

# Create one-vs-all comparison
pipeline.comparison.create_comparison(
    name="cluster_comparison",
    mode="one_vs_rest",
    feature_selector="contacts_only",
    data_selector_groups="cluster"
)

In [ ]:
pipeline.feature_importance.add.decision_tree(
    comparison_name="cluster_comparison",
    analysis_name="feature_importance",
)

In [ ]:
fig = pipeline.plots.feature_importance.decision_trees(
    feature_importance_name="feature_importance",
    save_fig=True,
    short_layout=True,
    hide_path=False,
    separate_trees=True,
    width_scale_factor=0.4,
    height_scale_factor=0.5,
    edge_symbol_fontsize=18,
    file_format="svg"
)

In [ ]:
fig = pipeline.plots.feature_importance.violins(
    feature_importance_name="feature_importance",
    n_top=2,
    save_fig=True,
    max_cols=7,
    file_format="svg",
    tick_fontsize=26,
    ylabel_fontsize=26,
    subplot_title_fontsize=26,
)

In [ ]:
fig = pipeline.plots.feature_importance.time_series(
    feature_importance_name="feature_importance",
    n_top=2,
    save_fig=True,
    max_cols=1,
    membership_per_feature=True,
    clustering_name="DPA_ContactKPC",
    file_format="svg",
    tick_fontsize=26,
    ylabel_fontsize=26,
    xlabel_fontsize=26,
    subplot_title_fontsize=26,
)

In [ ]:
pipeline.structure_visualization.feature_importance.create_pdb_with_beta_factors(
    structure_viz_name="structure_viz",
    feature_importance_name="feature_importance",
)
pipeline.structure_visualization.feature_importance.create_pymol_script(
    structure_viz_name="structure_viz"
)

In [ ]:
ui, view = pipeline.structure_visualization.feature_importance.visualize_nglview_jupyter(
    structure_viz_name="structure_viz",
)

In [ ]:
pipeline.save_to_single_file("cache/example_pipeline.pkl")

In [ ]:
loaded_pipeline = PipelineManager.load_from_single_file("cache/example_pipeline.pkl")